In [19]:
import pickle

import cv2
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt

from pathlib import Path

In [2]:
import utils
import calibration_utils

import importlib
importlib.reload(utils)
importlib.reload(calibration_utils)

<module 'calibration_utils' from '/home/tom/Documents/UNT/csce6260/projects/pcdet_notebooks/calibration_utils.py'>

In [3]:
CLASS_COLORS = {
    1: (1, 0, 0),    # Car     (red)
    2: (0, 1, 0),    # Pedestrian (green)
    3: (0, 0, 1)     # Cyclist (blue)
}

def get_point_colors(z):
    z_norm = (z - z.min()) / (z.max() - z.min() + 1e-6)
    colors = plt.cm.jet(z_norm)[:, :3]
    return colors

def create_point_cloud(points):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    colors = get_point_colors(points[:, 2])
    pcd.colors = o3d.utility.Vector3dVector(colors)
    return pcd

def get_min_and_max_values(points):
    min_x, max_x = np.min(points[:, 0]), np.max(points[:, 0])
    min_y, max_y = np.min(points[:, 1]), np.max(points[:, 1])
    min_z, max_z = np.min(points[:, 2]), np.max(points[:, 2])
    return min_x, max_x, min_y, max_y, min_z, max_z

def format_filename(sample_id, ext="txt"):
    return f"{sample_id:06d}.{ext}"

### Augmented Data

In [4]:
data_dir = Path.cwd() / "data"

In [5]:
sample_id = 0
sample_file_path = "sample_" + format_filename(sample_id, "pkl")
sample_file_path = data_dir / sample_file_path

with open(sample_file_path, "rb") as f:
    sample = pickle.load(f)

print("Sample keys:", sample.keys())

Sample keys: dict_keys(['frame_id', 'gt_boxes', 'points', 'flip_x', 'noise_rot', 'noise_scale', 'lidar_aug_matrix', 'use_lead_xyz', 'image_shape'])


In [6]:
points = sample['points']

print("Points shape:", points.shape)
print('Min/Max values for all dimensions')
min_x, max_x, min_y, max_y, min_z, max_z = get_min_and_max_values(points)
print(f"X: ({min_x:.2f}, {max_x:.2f})")
print(f"Y: ({min_y:.2f}, {max_y:.2f})")
print(f"Z: ({min_z:.2f}, {max_z:.2f})")

Points shape: (16384, 4)
Min/Max values for all dimensions
X: (5.25, 63.28)
Y: (-15.11, 29.17)
Z: (-2.43, 2.02)


In [7]:
def get_gt_oriented_bounding_boxes(gt_boxes):
    boxes = []
    for box in gt_boxes:
        x, y, z, dx, dy, dz, heading, class_idx = box
        class_idx = int(class_idx)
        obb = o3d.geometry.OrientedBoundingBox(
            center=[0, 0, 0],
            R=o3d.geometry.get_rotation_matrix_from_xyz([0, 0, heading]),
            extent=[dx, dy, dz]
        )
        obb.translate([x, y, z])
        obb.color = CLASS_COLORS.get(class_idx, (1, 1, 1))
        boxes.append(obb)
    return boxes


points = sample['points'][:, :3]
pcd = create_point_cloud(points)
gt_boxes = get_gt_oriented_bounding_boxes(sample['gt_boxes'])
o3d.visualization.draw_geometries([pcd, *gt_boxes])

### Original Data

In [16]:
base = Path.home() / "kitti"
train_dir = base / "training"
velo_dir = train_dir / "velodyne"
calib_dir = train_dir / "calib"
label_dir = train_dir / "label_2"
img_dir = train_dir/ "image_2"

In [9]:
bin_path = velo_dir / format_filename(sample_id, "bin")
orig_points = np.fromfile(bin_path, dtype=np.float32).reshape(-1, 4)

print("Points shape:", orig_points.shape)
print('Min/Max values for all dimensions')
min_x, max_x, min_y, max_y, min_z, max_z = get_min_and_max_values(orig_points)
print(f"X: ({min_x:.2f}, {max_x:.2f})")
print(f"Y: ({min_y:.2f}, {max_y:.2f})")
print(f"Z: ({min_z:.2f}, {max_z:.2f})")

Points shape: (115384, 4)
Min/Max values for all dimensions
X: (-71.04, 73.04)
Y: (-21.10, 53.80)
Z: (-5.16, 2.67)


In [10]:
def get_labels(label_file_path):
    labels = utils.parse_label_file(label_file_path)
    return labels

def get_Tr_cam_to_velo(calib_file_path):
    calib = utils.parse_calib_file(calib_file_path)
    Tr_velo_to_cam = calib['Tr_velo_to_cam']
    Tr_velo_to_cam = np.vstack((Tr_velo_to_cam, [0, 0, 0, 1]))
    Tr_cam_to_velo = calibration_utils.inverse_rigid_transform(Tr_velo_to_cam)
    return Tr_cam_to_velo

def get_gt_oriented_bounding_boxes(labels, Tr_cam_to_velo):
    gt_oriented_boxes = []
    for label in labels:
        corners_3d_velo = calibration_utils.compute_box_3d(label, Tr_cam_to_velo)
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(corners_3d_velo)
        oriented_bbox = pcd.get_oriented_bounding_box()
        oriented_bbox.color = (0, 1, 0)
        gt_oriented_boxes.append(oriented_bbox)
    return gt_oriented_boxes


orig_points = orig_points[:, :3]
orig_pcd = create_point_cloud(orig_points)
label_file_path = label_dir / format_filename(sample_id, "txt")
labels = get_labels(label_file_path)
calib_file_path = calib_dir / format_filename(sample_id, "txt")
Tr_cam_to_velo = get_Tr_cam_to_velo(calib_file_path)
orig_gt_boxes = get_gt_oriented_bounding_boxes(labels, Tr_cam_to_velo)
o3d.visualization.draw_geometries([orig_pcd, *orig_gt_boxes])

### Data Processing (mask/sample/shuffle)

### Field-of-View (FOV) Filtering

In [18]:
sample['image_shape']

array([ 370, 1224], dtype=int32)

In [20]:
img_path = img_dir / format_filename(sample_id, "png")
img_path

PosixPath('/home/tom/kitti/training/image_2/000000.png')

In [21]:
img = cv2.imread(str(img_path))
img_shape = img.shape
img_shape

(370, 1224, 3)

#### Mask Points

In [11]:
POINT_CLOUD_X_RANGE = 0, 70.4
POINT_CLOUD_Y_RANGE = -40, 40
POINT_CLOUD_Z_RANGE = -3, 1

def mask_points_outside_range(points):
    min_x, max_x = POINT_CLOUD_X_RANGE
    min_y, max_y = POINT_CLOUD_Y_RANGE
    mask = (points[:, 0] >= min_x) & (points[:, 0] <= max_x) \
           & (points[:, 1] >= min_y) & (points[:, 1] <= max_y)
    return points[mask]

def mask_boxes_outside_range(gt_boxes):
    min_xyz = np.array([POINT_CLOUD_X_RANGE[0], POINT_CLOUD_Y_RANGE[0], POINT_CLOUD_Z_RANGE[0]])
    max_xyz = np.array([POINT_CLOUD_X_RANGE[1], POINT_CLOUD_Y_RANGE[1], POINT_CLOUD_Z_RANGE[1]])
    return [box for box in gt_boxes if np.all(box.center >= min_xyz) and np.all(box.center <= max_xyz)]

orig_points_masked = mask_points_outside_range(orig_points)
print("Points shape:", orig_points_masked.shape)
print('Min/Max values for all dimensions')
min_x, max_x, min_y, max_y, min_z, max_z = get_min_and_max_values(orig_points_masked)
print(f"X: ({min_x:.2f}, {max_x:.2f})")
print(f"Y: ({min_y:.2f}, {max_y:.2f})")
print(f"Z: ({min_z:.2f}, {max_z:.2f})")

Points shape: (63082, 3)
Min/Max values for all dimensions
X: (-0.00, 59.86)
Y: (-21.10, 34.73)
Z: (-5.16, 1.94)


In [12]:
orig_pcd_masked = create_point_cloud(orig_points_masked)
orig_gt_boxes_masked = mask_boxes_outside_range(orig_gt_boxes)
o3d.visualization.draw_geometries([orig_pcd_masked, *orig_gt_boxes_masked])

#### Sample Points

In [13]:
MAX_POINTS = 16384

def exceeds_max_points(points):
    return points.shape[0] > MAX_POINTS

def get_mask_points_near(points):
    points_dist = np.linalg.norm(points, axis=1)
    mask_points_near = points_dist < 40.0
    return mask_points_near

def get_far_and_near_idx(points):
    mask_points_near = get_mask_points_near(points)
    far_idx = np.where(mask_points_near == 0)[0]
    near_idx = np.where(mask_points_near == 1)[0]
    return far_idx, near_idx

def is_far_sparse(far_points):
    return far_points < MAX_POINTS

def retain_far_and_sample_near(points):
    far_idx, near_idx = get_far_and_near_idx(points)
    num_sample = max(0, MAX_POINTS - len(far_idx))
    sampled_near = np.random.choice(near_idx, num_sample, replace=False)
    choice = np.concatenate((far_idx, sampled_near)) if len(far_idx) > 0 else sampled_near
    np.random.shuffle(choice)
    return points[choice]

def sample_random(points):
    choice = np.random.choice(len(points), MAX_POINTS, replace=False)
    np.random.shuffle(choice)
    return points[choice]

def sample_distance_or_random(points):
    far_idx, near_idx = get_far_and_near_idx(points)
    if is_far_sparse(len(far_idx)):
        sampled_points = retain_far_and_sample_near(points)
    else:
        sampled_points = sample_random(points)
    return sampled_points

def sample_first(points):
    choice = np.random.choice(len(points), MAX_POINTS, replace=False)
    np.random.shuffle(choice)
    return points[choice]

def sample_points(points):
    if exceeds_max_points(points):
        sampled_points = sample_distance_or_random(points)
    else:
        sampled_points = sample_first(points)
    return sampled_points

orig_points_sampled = sample_points(orig_points_masked)
print("Points shape:", orig_points_sampled.shape)

Points shape: (16384, 3)


In [14]:
orig_pcd_sampled = create_point_cloud(orig_points_sampled)
o3d.visualization.draw_geometries([orig_pcd_sampled, *orig_gt_boxes_masked])